In [ ]:
from nltk.corpus import stopwords
import csv
import spacy
import os
import pandas as pd
import time
import pickle
import nltk
import numpy as np

# Dataset 1/2

In [ ]:
# Choose the dataset (1 for culture dataset, 2 for diversity dataset)
dataset = 1

# Dataset switch
if dataset == 1:
    LDAModelTimeStamp = '07092020_113759' # Dataset1
    noOfTopicsFolder = 'noOfTopics_18'  # Dataset1
    datasetName = 'Dataset1'
    topicsToEliminate = []
elif dataset == 2:
    LDAModelTimeStamp = '08092020_083631'  # Dataset2
    noOfTopicsFolder = 'noOfTopics_23' # Dataset2
    datasetName = 'Dataset2'
    topicsToEliminate = ['topic12','topic20','topic22']

# LDA Model target location

In [ ]:
dir = 'dataOCM/02_LDA/LDA_Runs/03_Selected/' + LDAModelTimeStamp
topicsDocumentsDistribution = pickle.load(open(dir + '/' + noOfTopicsFolder + '/' +  'LDA_04_Fitting_fittedData_' + LDAModelTimeStamp + '.data', 'rb'))

# Making dataframe from pickle

In [ ]:
tDocDist_df = pd.DataFrame(topicsDocumentsDistribution)
new_columns = tDocDist_df.loc[0]
tDocDist_df.columns = new_columns

# Counting no. of topics

In [ ]:
noOfTopics = 0
for name in new_columns:
    if 'topic' in name:
        noOfTopics +=1

# Size of dataframe

In [ ]:
tDocDist_df = tDocDist_df[1:tDocDist_df.size]
seriesLen = tDocDist_df['RvComment'].size
print('size of dataframe is :', seriesLen)

# Grouping organization sectors

In [ ]:
conditions = [(tDocDist_df["OrgSector"] == 'Immobilien'),
              (tDocDist_df["OrgSector"] == 'Elektro/Elektronik'),
              (tDocDist_df["OrgSector"] == 'Sonstige Branchen'),
              (tDocDist_df["OrgSector"] == 'Maschinenbau'),
              (tDocDist_df["OrgSector"] == 'Handel'),
              (tDocDist_df["OrgSector"] == 'Internet'),
              (tDocDist_df["OrgSector"] == 'Medizin/Pharma'),
              (tDocDist_df["OrgSector"] == 'IT'),
              (tDocDist_df["OrgSector"] == 'Automobil'),
              (tDocDist_df["OrgSector"] == 'Industrie'),
              (tDocDist_df["OrgSector"] == 'Dienstleistung'),
              (tDocDist_df["OrgSector"] == 'Energie'),
              (tDocDist_df["OrgSector"] == 'Marketing/Werbung/PR'),
              (tDocDist_df["OrgSector"] == 'Nahrungsmittel/Landwirtschaft'),
              (tDocDist_df["OrgSector"] == 'Chemie'),
              (tDocDist_df["OrgSector"] == 'Finanz'),
              (tDocDist_df["OrgSector"] == 'Banken'),
              (tDocDist_df["OrgSector"] == 'Transport/Verkehr/Logistik'),
              (tDocDist_df["OrgSector"] == 'Telekommunikation'),
              (tDocDist_df["OrgSector"] == 'Bau/Architektur'),
              (tDocDist_df["OrgSector"] == 'Versicherung'),
              (tDocDist_df["OrgSector"] == 'Kunst/Kultur'),
              (tDocDist_df["OrgSector"] == 'Textil'),
              (tDocDist_df["OrgSector"] == ' Immobilien'),
              (tDocDist_df["OrgSector"] == 'Medien'),
              (tDocDist_df["OrgSector"] == 'Personalwesen & -beschaffung'),
              (tDocDist_df["OrgSector"] == ' Industrie'),
              (tDocDist_df["OrgSector"] == 'Sport/Beauty'),
              (tDocDist_df["OrgSector"] == 'Druck/Verpackung'),
              (tDocDist_df["OrgSector"] == 'Medizintechnik'),
              (tDocDist_df["OrgSector"] == 'Forschung/Entwicklung'),
              (tDocDist_df["OrgSector"] == 'Beratung/Consulting')
              ]

choices_NewOrgSector = ['Finance_Insurance_Real_Estate', 'Manufacturing', 'Others', 'Manufacturing', 'Retail_Trade',
                        'Services','Manufacturing','Manufacturing','Manufacturing','Manufacturing','Services',
                        'Transportation_and_Public_Utilities','Services','Agriculture_Forestry_Fishing','Manufacturing',
                        'Finance_Insurance_Real_Estate','Finance_Insurance_Real_Estate','Transportation_and_Public_Utilities',
                        'Transportation_and_Public_Utilities','Construction','Finance_Insurance_Real_Estate','Services',
                        'Manufacturing','Finance_Insurance_Real_Estate','Services','Services','Manufacturing',
                        'Services','Manufacturing','Manufacturing','Services','Services']
tDocDist_df["NewOrgSector"] = np.select(conditions, choices_NewOrgSector, default='NaN')
listOfSectors = list(set(choices_NewOrgSector))

# Looping through list of sectors and exporting Cumulative Topics Proportions to csv

In [ ]:
for sec in listOfSectors:
    df_sec = tDocDist_df[tDocDist_df['NewOrgSector']==sec]

    csvFileNameOut = 'dataOCM/02_LDA/Cumulative_Topics_Proportions_SectorWise/RQ3_' + datasetName + '_TopicProps_' + sec +'.csv'
    csvFileOut = open(csvFileNameOut, "w", newline='', encoding='utf-8')
    csv_out = csv.writer(csvFileOut, delimiter='|')

    topicsProportions = []
    for t in range(noOfTopics):
        topicsProportions_temp = []
        topicsProportions_temp.append('topic' + str(t))
        topicsProportions_temp.append(sum(df_sec['topic' + str(t)]))
        topicsProportions_temp.append(0)
        topicsProportions.append(topicsProportions_temp)

    sumOfAllinDenom = sum(topicsProportions[c][1] for c in range(len(topicsProportions)) if topicsProportions[c][0] not in topicsToEliminate)

    for t in range(noOfTopics):
        if (topicsProportions[t][0] not in topicsToEliminate):
            topicsProportions[t][2] = topicsProportions[t][1] / sumOfAllinDenom
            csv_out.writerow(topicsProportions[t])
        else:
            topicsProportions[t][2] = 'topic eliminated'
            csv_out.writerow(topicsProportions[t])